# 97%

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/phunc20/biblio/blob/main/people/aurelien_geron/homl/07-ensemble_learning/notebooks/04.12.half_binomial_sum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

In [ ]:
import sys

In [ ]:
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install matplotlib==3.6.1 numpy==1.23.1 \
         scikit-learn==1.1.2 scipy==1.9.0 tqdm==4.64.0

In this notebook, we simply want our computer to help us compute
$$
\newcommand{\nchoosek}[2]{\begin{pmatrix}#1\\#2\end{pmatrix}}
P(\text{# Heads} > \text{# Tails}) = \sum_{k=\lfloor \frac{n}{2} \rfloor + 1}^{n} \nchoosek{n}{k} p^{k}(1-p)^{n-k}
$$
when $n = 10000$ because it'll be painful if we are to compute it
manually by pencil and paper.

## Word-by-word implementation

In [1]:
import math
import traceback
from tqdm.auto import tqdm

In [2]:
#math.comb?

In [3]:
def get_binomial_sum(n, x, y, *, start=None, end=None):
    if end is None:
        end = n+1
    if start is None:
        start = n//2 + 1
    try:
        return sum(
            get_kth_binomial_term(x, y, n, k) \
            for k in tqdm(range(start, end))
        )
    except Exception:
        traceback.print_exc()

In [4]:
def get_proba_binomial_sum(n, p, *, start=None, end=None):
    assert 0 <= p <= 1
    return get_binomial_sum(n, p, 1-p, start=start, end=end)

In [5]:
def get_kth_binomial_term(x, y, n, k):
    return math.comb(n, k) * x**k * y**(n-k)

In [6]:
get_proba_binomial_sum(1000, 0.51)

  0%|          | 0/500 [00:00<?, ?it/s]

0.7260985557305037

In [7]:
get_proba_binomial_sum(10_000, 0.51)

  0%|          | 0/5000 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "/tmp/ipykernel_11794/1104494544.py", line 7, in get_binomial_sum
    return sum(
  File "/tmp/ipykernel_11794/1104494544.py", line 8, in <genexpr>
    get_kth_binomial_term(x, y, n, k) \
  File "/tmp/ipykernel_11794/3365766825.py", line 2, in get_kth_binomial_term
    return math.comb(n, k) * x**k * y**(n-k)
OverflowError: int too large to convert to float


Reading the error message, we're kind of able to guess that
`math.comb(n, k)` reaches too large an integer such that it exceeds
the largest 64-bit floating-point number.

The next cell is designed to confirm that guess.

In [8]:
n = 10_000
k = n//2
try:
    float(math.comb(n, k))
except Exception:
    traceback.print_exc()

Traceback (most recent call last):
  File "/tmp/ipykernel_11794/1688019946.py", line 4, in <module>
    float(math.comb(n, k))
OverflowError: int too large to convert to float


This is actually quite common if you have experience calculating $n$
choose $k$ -- it just grows exponentially.

## Maybe it's just `math` package's fault
Well, let's try another package's `comb` function then. Say, `scipy`'s.

In [9]:
import scipy

In [10]:
n = 10_000
k = n//2
try:
    n_choose_k = float(scipy.special.comb(n, k))
except Exception:
    traceback.print_exc()

In [11]:
n_choose_k

inf

In [12]:
def get_kth_binomial_term(x, y, n, k):
    return scipy.special.comb(n, k) * x**k * y**(n-k)

In [13]:
get_proba_binomial_sum(10_000, 0.51)

  0%|          | 0/5000 [00:00<?, ?it/s]

/tmp/ipykernel_11794/2892321558.py:2: RuntimeWarning: invalid value encountered in double_scalars
  return scipy.special.comb(n, k) * x**k * y**(n-k)


nan

Well, this is pretty much the same, only that `scipy` doesn't crash
with `OverflowError`.

## 1st remedy

If `math.comb(n, k)` grows exponentially, let's
- not compute it entirely
- combine the powers of probability, i.e. $p^k$ and $(1-p)^{n-k}$, into it to slow down the growth

In [14]:
import numpy as np

In [15]:
def get_kth_binomial_term(x, y, n, k):
    p, q = max(k, n-k), min(k, n-k)
    B = np.arange(n-q+1, n+1) / np.arange(1, q+1)
    res = np.prod(
        B,
        initial=x**k * y**(n-k),
    )
    return res

In [16]:
n = 10_000
k = n//2
p = 0.51
try:
    kth_term = get_kth_binomial_term(p, 1-p, n, k)
except Exception:
    traceback.print_exc()

In [17]:
kth_term

0.0

In [18]:
get_proba_binomial_sum(10_000, 0.51)

  0%|          | 0/5000 [00:00<?, ?it/s]

0.0

We seem to have gone to the other extreme -- Every term becomes so small that it's virtually zero. This must be wrong. `get_proba_binomial_sum(10_000, 0.51)` should not give `0`.

## 2nd remedy

This time let's try to avoid the floating-point overflow simply
by making use of logarithms:
$$
\begin{aligned}
\nchoosek{n}{k} x^{k} y^{n-k}
&= \exp \left(\ln \left(\nchoosek{n}{k} x^{k} y^{n-k} \right) \right) \\
&= \exp \left(
    \ln \left(\nchoosek{n}{k}\right)
    + k \ln x
    + \left(n-k \right) \ln y
  \right)
\end{aligned}
$$

In [19]:
def get_kth_binomial_term(x, y, n, k):
    A = math.log(math.comb(n,k))
    B = k * math.log(x)
    C = (n-k) * math.log(y)
    return math.exp(A+B+C)

In [20]:
n = 10_000
k = n//2
p = 0.51
try:
    kth_term = get_kth_binomial_term(p, 1-p, n, k)
except Exception:
    traceback.print_exc()

In [21]:
kth_term

0.0010793603893886128

In [22]:
get_proba_binomial_sum(10_000, 0.51)

  0%|          | 0/5000 [00:00<?, ?it/s]

0.9767182874802731

In [23]:
get_proba_binomial_sum(1000, 0.51)

  0%|          | 0/500 [00:00<?, ?it/s]

0.7260985557304759